In [37]:
import pandas as pd 
import matplotlib as mpl
import seaborn
import matplotlib.pyplot as plt
import re
from collections import defaultdict

card_df=pd.read_csv('./CARD_SUBWAY_MONTH_202505.csv',index_col=False)

card_df


,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20250501,분당선,야탑,17571,18765,20250504
1,20250501,8호선,석촌,6970,8042,20250504
2,20250501,분당선,서현,16548,17173,20250504
3,20250501,경부선,독산,8851,8507,20250504
4,20250501,경부선,수원,35103,38426,20250504
...,...,...,...,...,...,...
19134,20250531,과천선,선바위,6421,5197,20250603
19135,20250531,공항철도 1호선,영종,4658,5004,20250603
19136,20250531,공항철도 1호선,인천공항2터미널,10249,7316,20250603
19137,20250531,공항철도 1호선,인천공항1터미널,16395,15278,20250603


In [ ]:
data= card_df[card_df['노선명'] == '2호선']

data
#data_loc=data.iloc[0,2,1:10]

#print(data_loc)

#line_2_stations = line_2_data['역명'].unique()
third_column_values_by_name = line_2_data['역명']

NameError: name 'line_2_data' is not defined